# ansible_ioscfg_filters
Prototyping ``ansible_iosfcfg

In [228]:
# The line below can be ignored but I didn't set up my environment properly 
import sys ; sys.path.append('/home/mjuenemann/.virtualenvs/ciscoconfparse/lib/python3.6/site-packages')

In [229]:
import ciscoconfparse

In [230]:
ansible_net_config = """
!
hostname router01
!
tacacs-server host 192.0.2.34
tacacs-server key cheezit
!
interface Ethernet2/0
 description Unprotected interface, facing towards Internet
 ip address 192.0.2.14 255.255.255.240
 no ip unreachables
 ntp disable
 no mop enable
 mtu 900
!
interface Ethernet2/1
 description Protected interface, facing towards DMZ
 ip address 192.0.2.17 255.255.255.240
 no mop enable
 
ip access-list extended LIST1
 allow ip host 10.1.1.1 host 10.99.99.99
 allow ip host 10.9.9.9 host 10.99.99.99
 allow ip host 10.1.1.1 host 192.168.1.1
!
ip access-list extended LIST2
 allow ip host 10.1.1.1 host 10.99.99.99
 allow ip host 10.1.1.1 host 172.16.1.1
"""

In [231]:
def _parse_config(config):
    return ciscoconfparse.CiscoConfParse(config.split('\n'))

def ioscfg_lines(config, lineregex):
    cfg = _parse_config(config)
    return [ioscfgline.text for ioscfgline in cfg.find_objects(lineregex)]

def ioscfg_lines_with_child(config, parentregex, childregex):
    cfg = _parse_config(config)
    return [ioscfgline.text for ioscfgline in cfg.find_objects_w_child(parentregex, childregex)]
    
def ioscfg_lines_and_child(config, parentregex, childregex):
    import re
    cfg = _parse_config(config)
    return [(ioscfgline.text, [child.text for child in ioscfgline.children 
                               if re.search(childregex, child.text)], )
            for ioscfgline in cfg.find_objects_w_child(parentregex, childregex)]

def ioscfg_lines_without_child(config, parentregex, childregex):
    cfg = _parse_config(config)
    return [ioscfgline.text for ioscfgline in cfg.find_objects_wo_child(parentregex, childregex)]

def ioscfg_lines_with_children(config, parentregex, childregexes):
    cfg = _parse_config(config)
    return [ioscfgline.text for ioscfgline in cfg.find_objects_w_all_children(parentregex, childregexes)]

def ioscfg_lines_without_children(config, parentregex, childregexes):
    cfg = _parse_config(config)
    return [ioscfgline.text for ioscfgline in cfg.find_objects_w_missing_children(parentregex, childregexes)]

def ioscfg_lines_with_parents(config, parentregex, childregex):
    cfg = _parse_config(config)
    return [ioscfgline.text for ioscfgline in cfg.find_objects_w_parents(parentregex, childregex)]

class FilterModule(object):
    def filters(self):
        return {
            'ioscfg_lines': ioscfg_lines,
            'ioscfg_lines_with_child': ioscfg_lines_with_child,
            'ioscfg_lines_and_child': ioscfg_lines_and_child,
            'ioscfg_lines_without_child': ioscfg_lines_without_child,
            'ioscfg_lines_with_children': ioscfg_lines_with_children,
            'ioscfg_lines_without_children': ioscfg_lines_without_children,
            'ioscfg_lines_with_parents': ioscfg_lines_with_parents
        }


## ioscfg_lines(lineregex)
Return lines matching the ``lineregex``.

In [232]:
ioscfg_lines(ansible_net_config, r'^tacacs')

['tacacs-server host 192.0.2.34', 'tacacs-server key cheezit']

## ioscfg_lines_with_child(parentregex, childregex):
Return lines matching ``parentregex`` that have a child matching ``childregex``.

In [233]:
ioscfg_lines_with_child(ansible_net_config, r'^interface', r'no mop enable')

['interface Ethernet2/0', 'interface Ethernet2/1']

## ioscfg_lines_and_child(parentregex, childregex):
This works like ``ioscfg_lines_with_child()`` but returns tuples of parent lines lines matching ``childregex``.
One possible use case could be to identify all access control lists and rules that relate to a specific IP address.

In [234]:
ioscfg_lines_and_child(ansible_net_config, r'^ip access-list extended', r'10.99.99.99')

[('ip access-list extended LIST1',
  [' allow ip host 10.1.1.1 host 10.99.99.99',
   ' allow ip host 10.9.9.9 host 10.99.99.99']),
 ('ip access-list extended LIST2',
  [' allow ip host 10.1.1.1 host 10.99.99.99'])]

## ioscfg_lines_without_child(parentregex, childregex)
Return lines matching ``parentregex`` but not have a child in ``childregex``.

In [235]:
ioscfg_lines_without_child(ansible_net_config, r'^interface', r'no ip unreachables')

['interface Ethernet2/1']

## ioscfg_lines_with_children(parentregex, childregexes)
Return lines matching ``parentregex`` and have *all* children in ``childregexes``. 

In [236]:
ioscfg_lines_with_children(ansible_net_config, r'^interface', [r'no ip unreachables', r'no mop enable'])

['interface Ethernet2/0']

## ioscfg_lines_without_children(parentregex, childregexes)
Return lines matching ``parentregex`` that have *none* of the children in ``childregexes``.

**DOES THIS WORK?**

In [237]:
ioscfg_lines_without_children(ansible_net_config, r'^interface', [r'no ip unreachables', r'no mop enable'])

['interface Ethernet2/1']

## ioscfg_lines_with_parents(parentregex, childregex)

Return lines matching ``childregex`` that have a parent matching ``parentregex``.

In [238]:
ioscfg_lines_with_parents(ansible_net_config, r'^interface', r'no mop enable')

[' no mop enable', ' no mop enable']